In [1]:
!pip install langchain langchain_community openai==0.28
!pip install matplotlib pillow
!pip install -U duckduckgo-search

In [2]:
import os
import time
from bs4 import BeautifulSoup
import re
import requests
import warnings

import langchain
from langchain.llms import OpenAI
from langchain.agents import load_tools, AgentType, Tool, initialize_agent

In [3]:
import os
import time
from bs4 import BeautifulSoup
import re
import requests
import warnings

import langchain
from langchain.llms import OpenAI
from langchain.agents import load_tools, AgentType, Tool, initialize_agent

In [28]:
os.environ["OPENAI_API_KEY"] = 'ENTER YOUR API KEY HERE'

warnings.filterwarnings("ignore")

In [5]:
llm=OpenAI(temperature=0,
           model_name="gpt-4o-mini")

# Tools

**Finding the ticker symbol of the stock**

In [7]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.prompts import ChatPromptTemplate

def find_ticker_symbol(customer_question):

    ticker_symbol_schema = ResponseSchema(
        name="ticker_symbol",
        description= "Extract the ticker symbol for the stock mentioned in the text. If the ticker symbol for the given stock name cannot be identified or does not exist, return 'NOTFOUND'. \Ensure that the extracted ticker symbol is accurate and corresponds directly to the stock name provided."
    )

    output_parser = StructuredOutputParser.from_response_schemas([ticker_symbol_schema])
    format_instructions = output_parser.get_format_instructions()

    review_template = f"""\
    For the following text, extract the following information:

    ticker_symbol: Extract the ticker symbol for the stock mentioned in the text.\
     If the ticker symbol for the given stock name cannot be identified or does not exist, return 'NOTFOUND'. \
     Ensure that the extracted ticker symbol is accurate and corresponds directly to the stock name provided.
    text: {{text}}

    {{format_instructions}}
    """
    prompt_template = ChatPromptTemplate.from_template(review_template)

    messages = prompt_template.format_messages(text=customer_question, format_instructions=format_instructions)

    # Invoke the LLM with the formatted messages
    response = llm.invoke(messages)

    # Parse the output from the response
    try:
        output_dict = output_parser.parse(response)
        return output_dict.get("ticker_symbol", "NOTFOUND")
    except Exception as e:
        print(f"Error processing response: {e}")
        return "NOTFOUND"

#print(find_ticker_symbol("How is the APPLE stock doing today"))


**Function for finding the basic information of the stock**

In [8]:
import yfinance as yf

def basic_stock_info(customer_question):
  ticker_symbol = find_ticker_symbol(customer_question)
  if ticker_symbol == "NOTFOUND":
    return "Error processing stock name in the question"
  try:
    ticker = yf.Ticker(ticker_symbol)
  except Exception as e:
    return("Error processing response")

  info = ticker.info

  company_info = (
    f"Below are the Basic details of {info['longName']} ::\n"
    f"Company name: {info['longName']},\n"
    f"Current price: {info['currentPrice']},\n"
    f"Market cap: {info['marketCap']},\n"
    f"PE Ratio (Trailing): {info['trailingPE']},\n"
    f"Dividend Yield: {info['dividendYield'] * 100:.2f}%,\n"
    f"52-Week High: {info['fiftyTwoWeekHigh']} USD,\n"
    f"52-Week Low: {info['fiftyTwoWeekLow']} USD, \n"
    f"Forward PE Ratio: {info['forwardPE']},"
  )

  return company_info

#print(basic_stock_info("AAPL"))

**Function for finding any specfic information or details of the stock**

In [9]:
import yfinance as yf
import json

def specific_stock_inforatiom(customer_question):
  ticker_symbol = find_ticker_symbol(customer_question)
  if ticker_symbol == "NOTFOUND":
    return "Error processing stock name in the question"
  try:
    ticker = yf.Ticker(ticker_symbol)
  except Exception as e:
    return("Error processing response")

  details = str([
    str(ticker.info),
    ticker.recommendations.to_string(),
    ticker.balance_sheet.to_string(),
    ticker.financials.to_string(),
    ticker.cashflow.to_string(),
    ticker.actions.to_string(),
    ticker.sustainability.to_string(),
    ticker.institutional_holders.to_string(),
    ticker.mutualfund_holders.to_string(),
    ticker.major_holders.to_string()
  ])

  return details

#print(specific_stock_inforatiom("How is APPLE looking today"))

**Function for plotting the historic stock price**

In [10]:
import matplotlib.pyplot as plt

def generate_matplotlib_graph(customer_query, filename='graph.png'):
  ticker_symbol = find_ticker_symbol(customer_query)
  if ticker_symbol == "NOTFOUND":
    return "Error processing stock name in the question"
  try:
    ticker = yf.Ticker(ticker_symbol)
  except Exception as e:
    return("Error processing response")

  try:
    historical_data = ticker.history(period="max")
  except Exception as e:
    return "Error fetching historical data: " + str(e)

  plt.plot(historical_data.index, historical_data["Close"], linestyle='-', color='b')
  plt.ylabel("Price in USD")
  plt.title("Historic stock price")
  plt.grid(True)

  image_path = os.path.join("/content", filename)

  try:
    plt.savefig(image_path)
  except Exception as e:
    return "Error saving plot: " + str(e)
  finally:
        plt.close()

  return image_path
generate_matplotlib_graph("Apple stock")

'/content/graph.png'

**Finding the agent execution tool chain**

In [11]:
import re
def find_tools_chain(intermediate_string):
  # Define the patterns
  intermediate_string = str(intermediate_string)
  tool_pattern = re.compile(r"tool='([^']*)'")
  tool_input_pattern = re.compile(r"tool_input='([^']*)'")

  # Find the matchees
  tools = tool_pattern.findall(intermediate_string)
  tool_inputs = tool_input_pattern.findall(intermediate_string)

  tool_chain = " --> ".join(tools)

  return tool_chain
#find_tools_chain(response["intermediate_steps"])

In [12]:
import langchain
from langchain.llms import OpenAI
from langchain.agents import load_tools, AgentType, Tool, initialize_agent

In [13]:
llm.invoke("Hello")

'Hello! How can I assist you today?'

In [14]:
tools = [
    Tool(
        name = "find stock ticker symbol",
        func =  find_ticker_symbol ,
        description = "Use when you want to extract the ticker symbol of the stock"

    ),
    Tool(
        name = "specific stock information",
        func = specific_stock_inforatiom ,
        description = "Use when to find the specific details or information of the stock"
    ),
    Tool(
        name = "Historic stock plot",
        func = generate_matplotlib_graph ,
        description = "Use when you want to show or display the image path of image of historic stock price "
    )
]


In [15]:
from langchain.agents import initialize_agent

zero_shot_agent=initialize_agent(
    llm=llm,
    agent="zero-shot-react-description",
    tools=tools,
    verbose=False,
    max_iteration=4,
    return_intermediate_steps=True,
    handle_parsing_errors=True
)

In [16]:
print(zero_shot_agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

find stock ticker symbol(customer_question) - Use when you want to extract the ticker symbol of the stock
specific stock information(customer_question) - Use when to find the specific details or information of the stock
Historic stock plot(customer_query, filename='graph.png') - Use when you want to show or display the image path of image of historic stock price 

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [find stock ticker symbol, specific stock information, Historic stock plot]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [17]:
new_prompt = """You are a financial advisor Chatbot.Keep the reponses small and concise.You are a financial advisor tasked with
advising the users with the stock information.I

Answer the following questions as best you can .You have access to the following tools:

find stock ticker symbol(customer_question) - Use when you want to extract the ticker symbol of the stock
specific stock information(customer_question) - Use when to find the information or detials  of a stock
Historic stock plot(customer_query, filename='graph.png') - Use when you want to show or display the image path of image of historic stock price


Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [find stock ticker symbol, basic stock info]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

In [18]:
zero_shot_agent.agent.llm_chain.prompt.template=new_prompt

In [19]:
response = zero_shot_agent("How does apple stock looks like")
print(response["output"]), print("Execution tool chain ::\n ",find_tools_chain(response["intermediate_steps"]))

Apple Inc. (AAPL) is a leading technology company specializing in consumer electronics, with a current stock price of $218.36. It has a market cap of approximately $3.35 trillion, a dividend yield of 0.45%, and a trailing P/E ratio of 33.91. The stock is generally rated as a "buy" by analysts. For more details, visit [Apple's Investor Relations](http://investor.apple.com/).
Execution tool chain ::
  find stock ticker symbol --> specific stock information


(None, None)

In [20]:
response = zero_shot_agent("What is the ticker symbol of NVIDIA stock")
print(response["output"]), print("Execution tool chain ::\n ",find_tools_chain(response["intermediate_steps"]))

The ticker symbol for NVIDIA stock is NVDA.
Execution tool chain ::
  find stock ticker symbol


(None, None)

In [21]:
response = zero_shot_agent("Give me the basic details of the NANDA stock")
print(response["output"]), print("Execution tool chain ::\n ",find_tools_chain(response["intermediate_steps"]))

I couldn't find any information on NANDA stock. Please check the name or provide a different stock.
Execution tool chain ::
  find stock ticker symbol


(None, None)

In [22]:
response = zero_shot_agent("Give me last quarter  performance of the nvidia stock")
print(response["output"]), print("Execution tool chain ::\n ",find_tools_chain(response["intermediate_steps"]))

NVIDIA (NVDA) had a strong last quarter performance with a net income of approximately $29.76 billion and total revenue of about $60.92 billion.
Execution tool chain ::
  find stock ticker symbol --> specific stock information


(None, None)

In [23]:
response = zero_shot_agent("What is return on Assets of the nvidia stock")
print(response["output"]), print("Execution tool chain ::\n ",find_tools_chain(response["intermediate_steps"]))

The return on assets (ROA) for NVIDIA (NVDA) is approximately 49.1%.
Execution tool chain ::
  find stock ticker symbol --> specific stock information


(None, None)

In [24]:
response = zero_shot_agent("Give me a stock name which i should not invest with an example and why(Show the details)")
print(response["output"]), print("Execution tool chain ::\n ",find_tools_chain(response["intermediate_steps"]))

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/GME?modules=esgScores&corsDomain=finance.yahoo.com&formatted=false&symbol=GME&crumb=NuvJgWJhTla


GameStop Corp. (GME) is often advised against due to its high volatility, significant debt, and poor financial performance, including a trailing P/E ratio of 271.375 and negative revenue growth of -28.7%.
Execution tool chain ::
  find stock ticker symbol --> specific stock information


(None, None)

In [25]:
response = zero_shot_agent("What is the current price of the apple stock")
print(response["output"]), print("Execution tool chain ::\n ",find_tools_chain(response["intermediate_steps"]))

The current price of Apple stock (AAPL) is $218.36.
Execution tool chain ::
  find stock ticker symbol --> specific stock information


(None, None)

In [27]:
response = zero_shot_agent("Show me a plot of  apple stock price over years")
print(response["output"]), print("Execution tool chain ::\n ",find_tools_chain(response["intermediate_steps"]))

Here is the plot of Apple (AAPL) stock price over the years: /content/graph.png
Execution tool chain ::
  find stock ticker symbol --> Historic stock plot


(None, None)